# Image augmentation with geometric transformations

This notebook applies geometric transformations to images in the given directory

## Requirements
- the directory is structured in `images/` and `labels/`
- the images are in `.jpg` format
- the labels are in `.txt` format with YOLO format annotations
- for each training image, generate 3 outputs


## Preprocessing
- Stretch to `pixels` x `pixels`

## Augmentation
- Rotate (90, -90 or 180)
- Flip (horizontal or vertical)
- Brightness ([+`deltaBright`%, -`deltaBright`%])

In [41]:
# Config
PIXELS = 320
DELTA_BRIGHT = 50
OUTPUTS = 3

In [42]:
# Image transformations
from PIL import Image, ImageEnhance
import random
import os

def resize_image(image, pixels):
    return image.resize((pixels, pixels))

# resize the annotations of an image to resize as a square with length `pixels`
def resize_annotations(image, annotations, pixels):
    width, height = image.size
    new_width = pixels
    new_height = pixels

    # Calculate the scaling factors
    x_scale = new_width / width
    y_scale = new_height / height

    # Resize annotations
    resized_annotations = []
    for annotation in annotations:
        class_id, x_center, y_center, width, height = annotation
        x_center *= x_scale
        y_center *= y_scale
        width *= x_scale
        height *= y_scale
        resized_annotations.append((class_id, x_center, y_center, width, height))

    return annotations

def rotate_image(image, angle):
    return image.rotate(angle, expand=True)

def rotate_annotations(annotations, angle):
    # angle is only 90, -90 or 180
    rotated_annotations = []
    for annotation in annotations:
        class_id, x_center, y_center, width, height = annotation
        if angle == 90:
            new_x_center = y_center
            new_y_center = 1 - x_center
            new_width = height
            new_height = width
        elif angle == -90:
            new_x_center = 1 - y_center
            new_y_center = x_center
            new_width = height
            new_height = width
        elif angle == 180:
            new_x_center = 1 - x_center
            new_y_center = 1 - y_center
            new_width = width
            new_height = height
        else:
            raise ValueError("Angle must be 90, -90 or 180 degrees")
        
        rotated_annotations.append((class_id, new_x_center, new_y_center, new_width, new_height))
    return rotated_annotations

def flip_image(image, direction):
    if direction == 'horizontal':
        return image.transpose(Image.FLIP_LEFT_RIGHT)
    elif direction == 'vertical':
        return image.transpose(Image.FLIP_TOP_BOTTOM)
    else:
        raise ValueError("Direction must be 'horizontal' or 'vertical'")
    
def flip_annotations(annotations, direction):
    flipped_annotations = []
    for annotation in annotations:
        class_id, x_center, y_center, width, height = annotation
        if direction == 'horizontal':
            new_x_center = 1 - x_center
            new_y_center = y_center
        elif direction == 'vertical':
            new_x_center = x_center
            new_y_center = 1 - y_center
        else:
            raise ValueError("Direction must be 'horizontal' or 'vertical'")
        
        flipped_annotations.append((class_id, new_x_center, new_y_center, width, height))
    return flipped_annotations

def adjust_brightness(image, delta):
    # return the image with brightness adjusted by `delta` percent
    enhancer = ImageEnhance.Brightness(image)
    factor = 1 + delta / 100.0  # Convert percentage to a factor
    return enhancer.enhance(factor)

def process_image(image, annotations, pixels, angle, flip_direction, brightness_delta):
    # Resize the image
    resized_image = resize_image(image, pixels)
    resized_annotations = resize_annotations(image, annotations, pixels)

    # Rotate the image
    if angle is not None:
        rotated_image = rotate_image(resized_image, angle)
        rotated_annotations = rotate_annotations(resized_annotations, angle)
    else:
        rotated_image = resized_image
        rotated_annotations = resized_annotations

    # Flip the image
    if flip_direction is not None:
        flipped_image = flip_image(rotated_image, flip_direction)
        flipped_annotations = flip_annotations(rotated_annotations, flip_direction)
    else:
        flipped_image = rotated_image
        flipped_annotations = rotated_annotations

    # Adjust brightness
    if brightness_delta is not None:
        final_image = adjust_brightness(flipped_image, brightness_delta)
    else:
        final_image = flipped_image

    return final_image, flipped_annotations

def transform_image(image, annotations):
    output = {
        'images': [],
        'annotations': [],
        'transformations': []
    }

    pixels = PIXELS

    # Add the original image and annotations resized
    resized_image = resize_image(image, pixels)
    resized_annotations = resize_annotations(image, annotations, pixels)
    output['images'].append(resized_image)
    output['annotations'].append(resized_annotations)
    output['transformations'].append(('N', 'N', 'N'))  # Original image has no transformations

    # Prepare to generate transformations
    transformations = [
        ('N', 'N', 'N')  # Original image transformation
    ]
    # Generate three transformations randomly selected and check if they are all different
    angles = [90, -90, 180, None]  # None means no rotation
    flip_directions = ['horizontal', 'vertical', None]  # None means no flip
    brightness_max = [DELTA_BRIGHT, None]
    while len(transformations) < OUTPUTS:
        angle = random.choice(angles)
        flip_direction = random.choice(flip_directions)
        brightness_delta = random.choice(brightness_max)
        if brightness_delta is not None:
            # select a number between -DELTA_BRIGHT and DELTA_BRIGHT
            brightness_delta = round(random.uniform(-DELTA_BRIGHT, DELTA_BRIGHT), 2)

        # Check if the transformation is already in the list
        transformation = (angle, flip_direction, brightness_delta)
        if transformation not in transformations:
            transformations.append(transformation)
            final_image, final_annotations = process_image(image, annotations, pixels, angle, flip_direction, brightness_delta)
            output['images'].append(final_image)
            output['annotations'].append(final_annotations)
            # tranform None into N string
            angle = angle if angle is not None else 'N'
            flip_direction = flip_direction if flip_direction is not None else 'N'
            brightness_delta = brightness_delta if brightness_delta is not None else 'N'
            output['transformations'].append((angle, flip_direction, brightness_delta))
    
    return output

def load_transform_store(src_dir, image_name, out_dir):
    """
    Load an image and its annotations, apply transformations, and store the results.
    
    :param src_dir: Source directory containing images and labels
    :param image_name: Name of the image file with .jpg extension (without path)
    :param out_dir: Output directory to store transformed images and annotations
    :return: None
    """
    # Load the image
    image_path = os.path.join(src_dir, 'images', image_name)
    image = Image.open(image_path)

    # Load the annotations
    label_name = image_name.replace('.jpg', '.txt')
    label_path = os.path.join(src_dir, 'labels', label_name)
    with open(label_path, 'r') as f:
        annotations = [tuple(map(float, line.strip().split()[:5])) for line in f.readlines()]

    # Transform the image and annotations
    output = transform_image(image, annotations)

    # Store the results
    for i, (final_image, final_annotations, transformation) in enumerate(zip(output['images'], output['annotations'], output['transformations'])):
        # Save the image
        angle, flip_direction, brightness_delta = transformation
        if angle == 'N' and flip_direction == 'N' and brightness_delta == 'N':
            suffix = ''
        else:
            suffix = f"_{angle}_{flip_direction}_{brightness_delta}".replace(".", ",")
        final_image_name = image_name.replace('.jpg', f'{suffix}.jpg')
        final_image_path = os.path.join(out_dir, 'images', final_image_name)
        final_image.save(final_image_path)

        # Save the annotations
        final_label_name = final_image_name.replace('.jpg', '.txt')
        final_label_path = os.path.join(out_dir, 'labels', final_label_name)
        with open(final_label_path, 'w') as f:
            for annotation in final_annotations:
                class_id, x_center, y_center, width, height = annotation
                f.write(f"{int(class_id)} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
        print(f"Transformed image saved as: {final_image_path}")
        print(f"Transformed annotations saved as: {final_label_path}")

In [43]:
DATASETS_DIR = "../../datasets/extracted"

DATASET_SRC = "RoboFlow-2"
DATASET_DEST = DATASET_SRC + "-augmented"
DATASET_SET_FOLDER = "train"

src_dir = os.path.join(DATASETS_DIR, DATASET_SRC, DATASET_SET_FOLDER)
out_dir = os.path.join(DATASETS_DIR, DATASET_DEST, DATASET_SET_FOLDER)

In [44]:
# Remove the output directory if it exists
if os.path.exists(out_dir):
    print(f"Removing existing output directory: {out_dir}")
    for root, dirs, files in os.walk(out_dir, topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            os.rmdir(os.path.join(root, name))
    os.rmdir(out_dir)

# Create output directories if they do not exist
os.makedirs(os.path.join(out_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(out_dir, 'labels'), exist_ok=True)

# Process each image in the source directory
for image_name in os.listdir(os.path.join(src_dir, 'images')):
    if image_name.endswith('.jpg'):
        print(f"Processing image: {image_name}")
        load_transform_store(src_dir, image_name, out_dir)
    else:
        print(f"Skipping non-image file: {image_name}")
print("Data augmentation completed.")

Removing existing output directory: ../../datasets/extracted/RoboFlow-2-augmented/train
Processing image: 847963f0-R_3018_jpg.rf.ff9d79319ee5c2070abfd5a97222334c.jpg
Transformed image saved as: ../../datasets/extracted/RoboFlow-2-augmented/train/images/847963f0-R_3018_jpg.rf.ff9d79319ee5c2070abfd5a97222334c.jpg
Transformed annotations saved as: ../../datasets/extracted/RoboFlow-2-augmented/train/labels/847963f0-R_3018_jpg.rf.ff9d79319ee5c2070abfd5a97222334c.txt
Transformed image saved as: ../../datasets/extracted/RoboFlow-2-augmented/train/images/847963f0-R_3018_jpg.rf.ff9d79319ee5c2070abfd5a97222334c_90_vertical_N.jpg
Transformed annotations saved as: ../../datasets/extracted/RoboFlow-2-augmented/train/labels/847963f0-R_3018_jpg.rf.ff9d79319ee5c2070abfd5a97222334c_90_vertical_N.txt
Transformed image saved as: ../../datasets/extracted/RoboFlow-2-augmented/train/images/847963f0-R_3018_jpg.rf.ff9d79319ee5c2070abfd5a97222334c_-90_N_N.jpg
Transformed annotations saved as: ../../datasets/ex

KeyboardInterrupt: 